# Library Load

In [ ]:
# 필요한 PyTorch 라이브러리 불러오기
import os
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision.utils import save_image

import PIL
from PIL import Image
import matplotlib.pyplot as plt

import cv2
import numpy as np

import xml.etree.ElementTree as ET

import copy

In [ ]:
# GPU 장치 사용 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

# Data Load

In [ ]:
# Content Image Path
content_img_folder_path = '../data/1. original_img_normal/'
content_img_folder_list = os.listdir(content_img_folder_path)
content_img_folder_list

In [ ]:
# Style Image Path
style_img_path = '../data/style_img/KimHD.jpg'

In [ ]:
# 뉴럴 네트워크 모델을 불러옵니다.
cnn = models.vgg19(pretrained=True).features.to(device).eval()
print(cnn)

# Function

In [ ]:
# 이미지를 불러와 다운받아 텐서(Tensor) 객체로 변환하는 함수
def image_loader(img_path, imsize):
    loader = transforms.Compose([
        transforms.Resize(imsize), # 이미지의 크기를 변경
        transforms.ToTensor() # torch.Tensor 형식으로 변경 [0, 255] → [0, 1]
    ])
    image = PIL.Image.open(img_path)
    # 네트워크 입력에 들어갈 이미지에 배치 목적의 차원(dimension) 추가
    image = loader(image).unsqueeze(0)
    return image.to(device, torch.float) # GPU로 올리기


# torch.Tensor 형태의 이미지를 화면에 출력하는 함수
def imshow(tensor):
    # matplotlib는 CPU 기반이므로 CPU로 옮기기
    image = tensor.cpu().clone()
    # torch.Tensor에서 사용되는 배치 목적의 차원(dimension) 제거
    image = image.squeeze(0)
    # PIL 객체로 변경 
    image = transforms.ToPILImage()(image)
    # 이미지를 화면에 출력(matplotlib는 [0, 1] 사이의 값이라고 해도 정상적으로 처리)
    plt.imshow(image)
    plt.show()

In [ ]:
# 입력 정규화(Normalization)를 위한 초기화
cnn_normalization_mean = torch.tensor([0.485, 0.456, 0.406]).to(device)
cnn_normalization_std = torch.tensor([0.229, 0.224, 0.225]).to(device)

class Normalization(nn.Module):
    def __init__(self, mean, std):
        super(Normalization, self).__init__()
        self.mean = mean.clone().view(-1, 1, 1)
        self.std = std.clone().view(-1, 1, 1)

    def forward(self, img):
        return (img - self.mean) / self.std

In [ ]:
def gram_matrix(input):
    # a는 배치 크기, b는 특징 맵의 개수, (c, d)는 특징 맵의 차원을 의미
    a, b, c, d = input.size()
    # 논문에서는 i = 특징 맵의 개수, j = 각 위치(position)
    features = input.view(a * b, c * d)
    # 행렬 곱으로 한 번에 Gram 내적 계산 가능
    G = torch.mm(features, features.t())
    # Normalize 목적으로 값 나누기
    return G.div(a * b * c * d)


# 스타일 손실(style loss) 계산을 위한 클래스 정의
class StyleLoss(nn.Module):
    def __init__(self, target_feature):
        super(StyleLoss, self).__init__()
        self.target = gram_matrix(target_feature).detach()

    def forward(self, input):
        G = gram_matrix(input)
        self.loss = F.mse_loss(G, self.target)
        return input
    

# 콘텐츠 손실(content loss) 계산을 위한 클래스 정의
class ContentLoss(nn.Module):
    def __init__(self, target,):
        super(ContentLoss, self).__init__()
        self.target = target.detach()

    def forward(self, input):
        self.loss = F.mse_loss(input, self.target)
        return input

In [ ]:
content_layers = ['conv_2', 'conv_4']
style_layers = ['conv_1', 'conv_3', 'conv_5', 'conv_7', 'conv_9']

# Style Transfer 손실(loss)을 계산하는 함수
def get_losses(cnn, content_img, style_img, noise_image):
    cnn = copy.deepcopy(cnn)
    normalization = Normalization(cnn_normalization_mean, cnn_normalization_std).to(device)
    content_losses = []
    style_losses = []
    
    # 가장 먼저 입력 이미지가 입력 정규화(input normalization)를 수행하도록
    model = nn.Sequential(normalization)

    # 현재 CNN 모델에 포함되어 있는 모든 레이어를 확인하며
    i = 0
    for layer in cnn.children():
        if isinstance(layer, nn.Conv2d):
            i += 1
            name = 'conv_{}'.format(i)
        elif isinstance(layer, nn.ReLU):
            name = 'relu_{}'.format(i)
            layer = nn.ReLU(inplace=False)
        elif isinstance(layer, nn.MaxPool2d):
            name = 'pool_{}'.format(i)
        elif isinstance(layer, nn.BatchNorm2d):
            name = 'bn_{}'.format(i)
        else:
            raise RuntimeError('Unrecognized layer: {}'.format(layer.__class__.__name__))

        model.add_module(name, layer)

        # 설정한 content layer까지의 결과를 이용해 content loss를 계산
        if name in content_layers:
            target_feature = model(content_img).detach()
            content_loss = ContentLoss(target_feature)
            model.add_module("content_loss_{}".format(i), content_loss)
            content_losses.append(content_loss)

        # 설정한 style layer까지의 결과를 이용해 style loss를 계산
        if name in style_layers:
            target_feature = model(style_img).detach()
            style_loss = StyleLoss(target_feature)
            model.add_module("style_loss_{}".format(i), style_loss)
            style_losses.append(style_loss)

    # 마지막 loss 이후의 레이어는 사용하지 않도록
    for i in range(len(model) - 1, -1, -1):
        if isinstance(model[i], ContentLoss) or isinstance(model[i], StyleLoss):
            break

    model = model[:(i + 1)]
    return model, content_losses, style_losses

In [ ]:
def style_transfer(cnn, content_img, style_img, input_img, iters):
    model, content_losses, style_losses = get_losses(cnn, content_img, style_img, input_img)
    optimizer = optim.LBFGS([input_img.requires_grad_()])

    print("[ Start ]")
    imshow(input_img)

    # 하나의 값만 이용하기 위해 배열 형태로 사용
    run = [0]
    while run[0] <= iters:

        def closure():
            input_img.data.clamp_(0, 1)

            optimizer.zero_grad()
            model(input_img)
            content_score = 0
            style_score = 0

            for cl in content_losses:
                content_score += cl.loss
            for sl in style_losses:
                style_score += sl.loss

            style_score *= 1e5
            loss = content_score + style_score
            loss.backward()

            run[0] += 1
            if run[0] % 100 == 0:
                print(f"[ Step: {run[0]} / Content loss: {content_score.item()} / Style loss: {style_score.item()}]")
                imshow(input_img)
            
            return content_score + style_score
        
        optimizer.step(closure)

    # 결과적으로 이미지의 각 픽셀의 값이 [0, 1] 사이의 값이 되도록 자르기
    input_img.data.clamp_(0, 1)

    return input_img

# Style Transfer

In [ ]:
for content_img_folder_name in tqdm(content_img_folder_list):
    print(content_img_folder_name)
    # 폴더 내의 파일 리스트
    content_img_list = os.listdir(content_img_folder_path + content_img_folder_name)
    
    #imshow(style_img)
    for content_img_name in content_img_list:
        content_img_size = PIL.Image.open(content_img_folder_path + content_img_folder_name + '/' + content_img_name).size
        #con_arr = np.array(con_image)
        #con_tensor = torch.from_numpy(con_arr)
        #con_tensor = con_tensor.to(device)
        content_img = image_loader(content_img_folder_path + content_img_folder_name + '/' + content_img_name, content_img_size)
        #imshow(content_img)

        input_img = torch.empty_like(content_img).uniform_(0, 1).to(device)
        #imshow(input_img)
        
        style_img = image_loader(style_img_path, (content_img.shape[2], content_img.shape[3]))

        # style transfer 수행
        output = style_transfer(cnn, content_img=content_img, style_img=style_img, input_img=input_img, iters=1500)

        # 이미지 저장
        save_image(output.cpu().detach()[0], '../data/3. style_aug_img/' + content_img_folder_name + '/style_' + content_img_name)
        print('='*60)

# Bounding Box Change

In [ ]:
# Style Augmentation Image Path
style_aug_img_folder_path = '../data/3. style_aug_img/'
style_aug_img_folder_list = os.listdir(style_aug_img_folder_path)
style_aug_img_folder_list

In [ ]:
# Label Path
label_folder_path = '../data/1. original_label_xml_style/'
label_folder_list = os.listdir(label_folder_path)
label_folder_list

In [ ]:
# Bounding Box Transform Yolo Shape
def convert(size, box):
    dw = 1./size[0]
    dh = 1./size[1]
    x = (box[0] + box[1])/2.0
    y = (box[2] + box[3])/2.0
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return (x,y,w,h)

In [ ]:
for img_folder_name, label_folder_name in tqdm(zip(style_aug_img_folder_list, label_folder_list)):
    # 폴더 내의 파일 리스트
    img_list = os.listdir(style_aug_img_folder_path + img_folder_name)
    label_list = os.listdir(label_folder_path + label_folder_name)
    
    print(img_folder_name, len(img_list))
    print(label_folder_name, len(label_list))
    print('-' * 60)
    
    # 확장자 제거한 임시 레이블 리스트
    tmp_label_list = []
    for label_name in label_list:
        tmp_label = label_name.split('.')[0]
        tmp_label_list.append(tmp_label)
    
    # 레이블링 안된 이미지 제거
    new_img_list = []
    for img_name in img_list:
        tmp_img = img_name.split('.')[0]
        if tmp_img in tmp_label_list:
            new_img_list.append(img_name)
            
    print(img_folder_name, len(new_img_list))
    print(label_folder_name, len(label_list))
    print('=' * 60)
    print('=' * 60)
            
    # XML 파일 파싱
    object_label_list = []
    for label_name_2 in label_list:
        xml_name = label_folder_path + label_folder_name + '/' + label_name_2
        tree = ET.parse(xml_name)
        objects = tree.findall('./object')

        box_list = []
        for object_ in objects:
            x1 = float(object_.findall('./bndbox')[0].find('./xmin').text)
            y1 = float(object_.findall('./bndbox')[0].find('./ymax').text)
            x2 = float(object_.findall('./bndbox')[0].find('./xmax').text)
            y2 = float(object_.findall('./bndbox')[0].find('./ymin').text)
            tmp_box_list = [x1, y1, x2, y2]
            box_list.append(tmp_box_list)

        object_label_list.append(np.array(box_list))
    print('object_label_list', len(object_label_list))
        
    # 이미지 불러오기
    for img_name_2, object_label in zip(new_img_list, object_label_list):
        load_img = cv2.imread(style_aug_img_folder_path + img_folder_name + '/' + img_name_2)[:,:,::-1]
        #plotted_img = draw_rect(load_img, object_label)
        #plt.imshow(plotted_img)
        #plt.show()
        
        # 레이블 수정
        y_ = load_img.shape[0]
        x_ = load_img.shape[1]
        targetSize = 200
        x_scale = targetSize / x_
        y_scale = targetSize / y_
        #print(x_scale, y_scale)
        
        # 레이블 저장
        im = Image.open(style_aug_img_folder_path + img_folder_name + '/' + img_name_2)
        w = int(im.size[0])
        h = int(im.size[1])
        
        f = open('../data/3. style_aug_label/' + label_folder_name + '/style_' + img_name_2[:-4] + '.txt', 'w')
        for box_2 in object_label:            
            # original frame as named values
            (origLeft, origTop, origRight, origBottom) = (box_2[0], box_2[2], box_2[1], box_2[3])
            x = int(np.round(origLeft * x_scale))
            y = int(np.round(origTop * y_scale))
            xmax = int(np.round(origRight * x_scale))
            ymax = int(np.round(origBottom * y_scale))
            change_bb = (x, y, xmax, ymax)
            
            bb = convert((w,h), change_bb)
            if img_folder_name[:-1] == 'carp':
                f.write(str(0)+' ')
                f.write(str(bb[0])+' ')
                f.write(str(bb[1])+' ')
                f.write(str(bb[2])+' ')
                f.write(str(bb[3])+'\n')
                
            elif img_folder_name[:-1] == 'cranebird':
                f.write(str(1)+' ')
                f.write(str(bb[0])+' ')
                f.write(str(bb[1])+' ')
                f.write(str(bb[2])+' ')
                f.write(str(bb[3])+'\n')
                
            elif img_folder_name[:-1] == 'deer':
                f.write(str(2)+' ')
                f.write(str(bb[0])+' ')
                f.write(str(bb[1])+' ')
                f.write(str(bb[2])+' ')
                f.write(str(bb[3])+'\n')
                
            elif img_folder_name[:-1] == 'lotusflower':
                f.write(str(3)+' ')
                f.write(str(bb[0])+' ')
                f.write(str(bb[1])+' ')
                f.write(str(bb[2])+' ')
                f.write(str(bb[3])+'\n')
                
            else:
                f.write(str(4)+' ')
                f.write(str(bb[0])+' ')
                f.write(str(bb[1])+' ')
                f.write(str(bb[2])+' ')
                f.write(str(bb[3])+'\n')
        f.close()

In [ ]:
def drawBox(boxes, image):
    for i in range(0, len(boxes)):
        # changed color and width to make it visible
        cv2.rectangle(image, (boxes[i][1], boxes[i][2]), (boxes[i][3], boxes[i][4]), (255, 0, 0), 1)
    cv2.imshow("img", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [ ]:
img = cv2.resize(load_img, (targetSize, targetSize));
img = np.array(img)
drawBox([[4, x, y, xmax, ymax]], img)